In [8]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
data = []
labels = []
for _ in range(10000):
    series = np.random.randint(0,9,30)
    data.append(series)
    if(sum(series)>=100):
        labels.append([0,1])
    else:
        labels.append([1,0])
train_input = data[0:8000]
test_input = data[8000:10000]
train_output = labels[0:8000]
test_output = labels[8000:10000]

In [9]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
serieses = []
labels = []
for _ in range(10000):
    series = np.random.randint(0,9,30)
    serieses.append(series)
    if(sum(series)>=100):
        labels.append([0,1])
    else:
        labels.append([1,0])
train_input = serieses[0:8000]
test_input = serieses[8000:10000]
train_output = labels[0:8000]
test_output = labels[8000:10000]
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 100
display_step = 200

# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 30 # timesteps
num_hidden = 200 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)
X = tf.placeholder("float", [None, 30, 1])
Y = tf.placeholder("float", [None, 2])
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(0, 80):
        batch_x, batch_y = np.array(train_input[step*100:(step+1)*100]),np.array(train_output[step*100:(step+1)*100])
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    
    test_data = np.array(test_input)
    test_data = test_data.reshape((batch_size, timesteps, num_input))
    test_label = np.array(test_output)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-ee072a0e572d>", line 50, in RNN
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
  File "<ipython-input-2-ee072a0e572d>", line 55, in <module>
    logits = RNN(X, weights, biases)
  File "/Applications/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
